In [3]:
import nltk
import pandas as pd
import numpy as np
import seaborn as sns

import sys
sys.path.append('../')

In [4]:
train = pd.read_csv('../data/train.csv')
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
train.sample(5)

,id,keyword,location,text,target
3516,5027,eyewitness,"Terlingua, Texas",#ClimateChange Eyewitness to Extreme Weather: ...,1
7335,10500,wildfire,Worldwide,California wildfire destroys more homes but cr...,1
4563,6486,injuries,"Orlando,FL USA",Official kinesiology tape of IRONMANå¨ long-la...,0
4963,7074,meltdown,Storybrooke / The Moors,A back to school commercial came on and my sis...,0
5764,8226,riot,United Kingdom,'Without an ally near you can't use this skill...,0
